In [4]:
import sys, os
import pandas as pd
import numpy
import pickle
from pathlib import Path
sys.path.append('../..')
from Utils import *

In [44]:
pickle_path = Path(Path('.').absolute()).parent.parent.absolute()/"Utils/Pickle"
movieId2poster_path = pickle_path / 'movieid_to_poster_file.pickle'


with open(movieId2poster_path,'rb') as f:
    movieId_to_posterfile = pickle.load(f)


character_df = pd.read_pickle(pickle_path / '230203_character_movie_merge.pickle')
movie_df = pd.read_pickle(pickle_path / '230130_Popular_movie_1192_cwj.pickle')
watch_link =  pd.read_pickle(pickle_path / '230131_watch_link_4679_rows.pickle')
engram_sim = pd.read_pickle(pickle_path / 'enneagram_similarity_075_099.pickle')

## 목표

    prefer movie ids -> rec movie ids -> mbti 필터링 -> interaction, side info 따로 DB에 저장
    index 페이지에서도 결과 보여주기

## 주피터에서 할일

    - prefer movie ids에서 interaction, side-info에 대한 rec movie ids 받고
    - rec movie ids에서 나와 유사한, 나 와 잘맞는 MBTI 캐릭터 필터링
    - 애니어그램 유사도 적용
    - 내림차순하여 context 리스트로 만들기

In [33]:
prefer_movie_ids = [115713, 91653, 111622, 80463, 197711, 1172, 300209, 81591]

In [34]:
interaction_prefer_movie_ids = [i for i in prefer_movie_ids if i < 300_000]
side_info_prefer_movie_ids = [i for i in prefer_movie_ids if i >= 300_000]

In [35]:
len(interaction_prefer_movie_ids), len(side_info_prefer_movie_ids)

(7, 1)

In [36]:
top_k=10
sideinfo_movie_list,scores = zip(*content_based_filtering(side_info_prefer_movie_ids, top_k))
sideinfo_movie_list

(300209, 300335, 138036, 300098, 176935, 160674, 2587, 164179, 300654, 99273)

In [38]:
interaction_movie_list = model_recommend_movies(interaction_prefer_movie_ids, top_k)
interaction_movie_list

/opt/ml/project/Utils/../interaction_model/LightGCN_64
10


[97921, 105355, 96821, 134393, 118900, 86882, 94959, 95873, 44555, 3418]

In [47]:
user_mbti = "INTJ"
user_enn = '1w9'

In [46]:
get_en_sim(user_enn,engram_sim)

{'1w9': 0.99,
 '1w2': 0.8600571270043009,
 '2w1': 0.767798564966392,
 '2w3': 0.7517287497838617,
 '3w2': 0.7942160971855593,
 '3w4': 0.8170706465981564,
 '4w3': 0.7546855319453264,
 '4w5': 0.7696335465492191,
 '5w4': 0.8244548190157155,
 '5w6': 0.8674300432273276,
 '6w5': 0.9199934004919502,
 '6w7': 0.7652736727326449,
 '7w6': 0.7497725865278099,
 '7w8': 0.7509678065546699,
 '8w7': 0.8058175852958114,
 '8w9': 0.8251301505755028,
 '9w8': 0.74478806867785,
 '9w1': 0.8406499535326831}

In [59]:
character_df.Enneagram.value_counts()

6w5    285
7w8    239
8w7    237
9w1    207
7w6    202
3w4    160
4w5    160
6w7    160
9w8    146
8w9    143
1w2    129
5w6    128
5w4    120
1w9    118
3w2    114
4w3    110
2w3    106
2w1    103
Name: Enneagram, dtype: int64

In [58]:
len(character_df.Enneagram.value_counts().tolist())

18

In [50]:
character_movie_id = character_df[character_df.CharacterId==1287].movieId.iloc[0]

In [52]:
char_df = character_df[character_df.movieId==character_movie_id]

In [60]:
char_df['Enneagram_sim'] = char_df.Enneagram.map(get_en_sim(user_enn,engram_sim))

In [61]:
char_df

,CharacterId,Character,vote,MBTI,Enneagram,movieId,contents_year,Contents,ko_title,rating,npop,img_src,Enneagram_sim
65,1287,Charlie Kelmeckis,1700,INFP,9w1,96821,2012,The Perks of Being a Wallflower (2012),월플라워,3.8,300000,https://static1.personality-database.com/profi...,0.840650
303,2899,Patrick Stewart,514,ENFP,7w6,96821,2012,The Perks of Being a Wallflower (2012),월플라워,3.8,300000,https://static1.personality-database.com/profi...,0.749773
366,14682,"Samantha ""Sam"" Button",386,ESFJ,2w3,96821,2012,The Perks of Being a Wallflower (2012),월플라워,3.8,300000,https://static1.personality-database.com/profi...,0.751729


In [62]:
mbti_list=["adf"]

In [66]:
char_df.loc[char_df.MBTI.isin(mbti_list), 'Enneagram_sim'] -= 15

In [68]:
user_fit_MBTI="INFP"

In [69]:
char_df.loc[char_df.MBTI==user_fit_MBTI, 'Enneagram_sim']

65    0.84065
Name: Enneagram_sim, dtype: float64

In [70]:
char_cols=['CharacterId','Character','img_src','ko_title','MBTI','hashtag','Enneagram_sim']

In [67]:
char_df

,CharacterId,Character,vote,MBTI,Enneagram,movieId,contents_year,Contents,ko_title,rating,npop,img_src,Enneagram_sim
65,1287,Charlie Kelmeckis,1700,INFP,9w1,96821,2012,The Perks of Being a Wallflower (2012),월플라워,3.8,300000,https://static1.personality-database.com/profi...,0.840650
303,2899,Patrick Stewart,514,ENFP,7w6,96821,2012,The Perks of Being a Wallflower (2012),월플라워,3.8,300000,https://static1.personality-database.com/profi...,0.749773
366,14682,"Samantha ""Sam"" Button",386,ESFJ,2w3,96821,2012,The Perks of Being a Wallflower (2012),월플라워,3.8,300000,https://static1.personality-database.com/profi...,0.751729


In [30]:
sideinfo_movie_list

(138036, 168610, 142488, 162606, 140174, 166486, 203322, 156605, 2256, 136562)